# МТС Geohack.112

## Данные

В таблице `zones.csv` записаны квадраты, примерный размер — 500х500 метров. Все квадраты расположены в Москве, либо на небольшом расстоянии от Москвы. Квадрат задается координатами нижнего левого угла `(lat_bl, lon_bl)` и верхнего правого `(lat_tr, lon_tr)`. В колонках `(lat_c, lon_c)` — координаты центра квадрата.


Квадраты, расположенные в западной части выборки, предназначены для обучения модели — для этих квадратов известно **среднее число вызовов экстренных служб** из квадрата в день:
- `calls_daily`: по всем дням
- `calls_workday`: по рабочим дням
- `calls_weekend`: по выходным дням
- `calls_wd{D}`: по дню недели `D` (0 — понедельник, 6 — воскресенье)

На квадратах из восточной части выборки необходимо построить прогноз числа вызовов по всем дням недели. Оцениваться качество предсказания будет не по всем квадратам, а по подмножеству, в которое **не** входят квадраты, вызовы из которых поступают крайне редко. Подмножество **целевых** квадратов имеет `is_target=1` в таблице. Для тестовых квадратов значения `calls_*` и `is_target` скрыты.


На карте обозначены квадраты трех типов:
- <span style="color: green;">Зеленые</span> — из обучающей части, не целевые
- <span style="color: red;">Красные</span> — из обучающей части, целевые
- <span style="color: blue;">Синие</span> — тестовые, на них необходимо построить прогноз


<img src="geohack_zones.png" width="300">

In [1]:
import pandas

df_zones = pandas.read_csv('data/zones.csv', index_col='zone_id')
df_zones.head()

,lat_bl,lon_bl,lat_tr,lon_tr,lat_c,lon_c,is_test,is_target,calls_daily,calls_workday,calls_weekend,calls_wd0,calls_wd1,calls_wd2,calls_wd3,calls_wd4,calls_wd5,calls_wd6
zone_id,,,,,,,,,,,,,,,,,,
0,55.378220,36.999105,55.382744,37.007050,55.380482,37.003077,0,0.0,0.002177,0.001325,0.004213,0.003006,0.000000e+00,6.885802e-04,0.003006,0.000000e+00,0.001374,0.007052
1,55.378252,37.006994,55.382775,37.014938,55.380514,37.010966,0,0.0,0.000996,0.000535,0.002096,0.000765,0.000000e+00,1.176913e-03,0.000765,0.000000e+00,0.001374,0.002818
2,55.378284,37.014883,55.382806,37.022827,55.380545,37.018855,0,0.0,0.001284,0.000724,0.002622,0.001322,0.000000e+00,1.020885e-03,0.001322,0.000000e+00,0.001374,0.003871
3,55.378315,37.022772,55.382837,37.030715,55.380576,37.026744,0,0.0,0.000968,0.000476,0.002143,0.000815,0.000000e+00,7.802124e-04,0.000815,0.000000e+00,0.001374,0.002912
4,55.378345,37.030662,55.382867,37.038603,55.380606,37.034633,0,0.0,0.000842,0.000310,0.002113,0.000783,1.821651e-07,7.286604e-07,0.000782,8.602241e-07,0.001374,0.002851


## Оценка качества

В качестве решения необходимо предоставить CSV таблицу с предсказаниями для всех тестовых квадратов, для каждого квадрата — по всем дням недели.

|zone_id|calls_wd0|calls_wd1|calls_wd2|calls_wd3|calls_wd4|calls_wd5|calls_wd6|
|-------|---------|---------|---------|---------|---------|---------|---------|
| 79    | 0.825861| 0.670869|0.786908 | 0.598091| 1.247591| 0.675773| 0.633927|
| ...   | ... ||

Качество оценивается только по подмножеству **целевых** квадратов. Участникам неизвестно, какие из квадратов целевые, однако принцип выбора целевых квадратов в обучающей и тестовой части — идентичен. 

Во время соревнования качество оценивается на 30% тестовых целевых квадратов (выбраны случайно), в конце соревнования итоги подводятся по оставшимся 70% квадратов.

Метрика качества предсказаний — коэффициент ранговой корреляции Кендалла ([Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient)), считается как доля пар объектов с неправильно упорядоченными предсказаниями:

$$\text{Kendall-}\tau = \frac{1}{n(n-1)} \sum_{i \neq j} \text{sgn}(y_i-y_j) \cdot \text{sgn}(\hat{y_j}-\hat{y_j})$$

Метрика оценивает порядок, в котором предсказания соотносятся друг с другом, а не их точные значения. Разные дни недели считаются независимыми элементами выборки, т.е. коэффициент корреляции считается по предсказаниям для всех тестовых пар `(zone_id, день недели)` (см. далее пример оценки качества на валидации).

В тестирующей системе используется реализация Kendall's tau из пакета SciPy: [`scipy.stats.kendalltau`](https://docs.scipy.org/doc/scipy-0.19.1/reference/generated/scipy.stats.kendalltau.html).

## Работа с данными OpenStreetMap

Организаторы не предоставляют никакой содержательной информации о квадратах (за исключением координат). Участникам необходимо самостоятельно искать информацию для предсказания из внешних источников. Разрешено использовать все публично доступные наборы данных, допускающие использование в соревновании, после того как этот набор данных был публично опубликован на форуме.

В качестве примера покажем, как можно построить признаковое описание квадратов на основе карты [OpenStreetMap](https://en.wikipedia.org/wiki/OpenStreetMap).

Актуальную выгрузку данных OpenStreetMap можно взять с сайта [GIS-Lab.info](http://gis-lab.info/projects/osm_dump/). Данные распространяются по лицензии [ODbL](https://wiki.openstreetmap.org/wiki/Open_Database_License), допускающей свободное использование при условии указания в явном виде источника: «© участники OpenStreetMap».

Загрузите файл [RU-MOS.osm.pbf](http://data.gis-lab.info/osm_dump/dump/latest/RU-MOS.osm.pbf) — это выгрузка карты по Московской области в компактном формате `osm.pbf`. Для чтения этих данных есть простая библиотека [`osmread`](https://pypi.python.org/pypi/osmread).

Карта OSM представляет собой набор [элементов](https://wiki.openstreetmap.org/wiki/Elements) трех видов:
1. **Node**: точки на карте
2. **Way**: дороги, площади, задаются набором точек
3. **Relation**: связи между элементами, например объединение дороги из нескольких частей

Элементы могут иметь набор тегов — пар ключ-значение.

##### Прочитаем только точки имеющие теги из нужного нам региона
Чтение может занять 10–30 минут.

In [2]:
%%time

import osmread
from tqdm import tqdm_notebook

LAT_MIN, LAT_MAX = 55.309397, 56.13526
LON_MIN, LON_MAX = 36.770379, 38.19270

osm_file = osmread.parse_file('osm/RU-MOS.osm.pbf')
tagged_nodes = [
    entry
    for entry in tqdm_notebook(osm_file, total=18976998)
    if isinstance(entry, osmread.Node)
    if len(entry.tags) > 0
    if (LAT_MIN < entry.lat < LAT_MAX) and (LON_MIN < entry.lon < LON_MAX)
]

HBox(children=(IntProgress(value=0, max=18976998), HTML(value='')))


CPU times: user 13min 45s, sys: 8.19 s, total: 13min 54s
Wall time: 14min 9s


In [3]:
import pickle

with open('osm/tagged_nodes.pickle', 'wb') as fout:
    pickle.dump(tagged_nodes, fout, protocol=pickle.HIGHEST_PROTOCOL)

##### Пример точки

In [4]:
tagged_nodes[0]

Node(id=1000, version=4, changeset=22527643, timestamp=1400951253, uid=109327, tags={'name': 'Десяточка', 'shop': 'supermarket', 'comment': 'ООО "АСП-ГРУПП", ИНН 7735108290, до 22:45 c 2014-05-23', 'opening_hours': '08:30-22:45'}, lon=37.2074648, lat=55.9950817)

## Построение таблицы признаков

На основе координат квадратов и данных из OpenStreetMap построим таблицу с простыми признаками, которые можно использовать для предсказания.

In [5]:
import collections 

df_features = collections.OrderedDict([])

##### Расстояние до Кремля

In [6]:
import math

kremlin_lat, kremlin_lon = 55.753722, 37.620657

def dist_calc(lat1, lon1, lat2, lon2):
    R = 6373.0

    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    return R * c

df_features['distance_to_kremlin'] = df_zones.apply(
    lambda row: dist_calc(row.lat_c, row.lon_c, kremlin_lat, kremlin_lon), axis=1)

##### Статистика по точкам из OSM

In [7]:
import numpy as np
from sklearn.neighbors import NearestNeighbors

# набор фильтров точек, по которым будет считаться статистика
POINT_FEATURE_FILTERS = [
    ('tagged', lambda 
     node: len(node.tags) > 0),
    ('railway', lambda node: node.tags.get('railway') == 'station'),
    ('shop', lambda node: 'shop' in node.tags),
    ('public_transport', lambda node: 'public_transport' in node.tags),
]

# центры квадратов в виде матрицы
X_zone_centers = df_zones[['lat_c', 'lon_c']].as_matrix()

for prefix, point_filter in POINT_FEATURE_FILTERS:

    # берем подмножество точек в соответствии с фильтром
    coords = np.array([
        [node.lat, node.lon]
        for node in tagged_nodes
        if point_filter(node)
    ])

    # строим структуру данных для быстрого поиска точек
    neighbors = NearestNeighbors().fit(coords)
    
    # признак вида "количество точек в радиусе R от центра квадрата"
    for radius in [0.001, 0.003, 0.005, 0.007, 0.01]:
        dists, inds = neighbors.radius_neighbors(X=X_zone_centers, radius=radius)
        df_features['{}_points_in_{}'.format(prefix, radius)] = np.array([len(x) for x in inds])

    # признак вида "расстояние до ближайших K точек"
    for n_neighbors in [3, 5, 10]:
        dists, inds = neighbors.kneighbors(X=X_zone_centers, n_neighbors=n_neighbors)
        df_features['{}_mean_dist_k_{}'.format(prefix, n_neighbors)] = dists.mean(axis=1)
        df_features['{}_max_dist_k_{}'.format(prefix, n_neighbors)] = dists.max(axis=1)
        df_features['{}_std_dist_k_{}'.format(prefix, n_neighbors)] = dists.std(axis=1)

    # признак вида "расстояние до ближайшей точки"
    df_features['{}_min'.format(prefix)] = dists.min(axis=1)

##### Итоговый набор признаков по квадратам

In [8]:
df_features = pandas.DataFrame(df_features, index=df_zones.index)
df_features.to_csv('data/features.csv')
df_features.head()

,distance_to_kremlin,tagged_points_in_0.001,tagged_points_in_0.003,tagged_points_in_0.005,tagged_points_in_0.007,tagged_points_in_0.01,tagged_mean_dist_k_3,tagged_max_dist_k_3,tagged_std_dist_k_3,tagged_mean_dist_k_5,...,public_transport_mean_dist_k_3,public_transport_max_dist_k_3,public_transport_std_dist_k_3,public_transport_mean_dist_k_5,public_transport_max_dist_k_5,public_transport_std_dist_k_5,public_transport_mean_dist_k_10,public_transport_max_dist_k_10,public_transport_std_dist_k_10,public_transport_min
zone_id,,,,,,,,,,,,,,,,,,,,,
0,56.852292,0,1,2,2,2,0.006777,0.015403,0.006120,0.011432,...,0.077722,0.100775,0.016301,0.089326,0.106988,0.019011,0.102931,0.119075,0.019160,0.066160
1,56.511876,0,0,0,0,2,0.008668,0.010560,0.001347,0.012681,...,0.077499,0.093368,0.011221,0.090663,0.110552,0.018316,0.104594,0.124896,0.019435,0.069525
2,56.173822,0,0,0,0,0,0.013769,0.015696,0.002323,0.017069,...,0.077767,0.086046,0.005854,0.089228,0.106559,0.014751,0.106627,0.130933,0.020701,0.073586
3,55.838172,0,0,0,0,0,0.020577,0.023549,0.003754,0.023259,...,0.078462,0.078834,0.000265,0.088230,0.103018,0.011964,0.108593,0.133372,0.022350,0.078232
4,55.504970,0,0,0,0,0,0.026699,0.030437,0.003545,0.028451,...,0.079531,0.083459,0.005491,0.087655,0.099975,0.010822,0.109584,0.137651,0.023498,0.071765


## Предсказание числа вызовов

#### Пример случайного разбиения на обучение / валидацию

In [9]:
from sklearn.model_selection import train_test_split

df_zones_train = df_zones.query('is_test == 0 & is_target == 1')
idx_train, idx_valid = train_test_split(df_zones_train.index, test_size=0.3)

#### Обучение модели

Обучаем `RandomForestRegressor` предсказывать среднее дневное число звонков, независимо от дня недели. День недели тем не менее можно учитывать.

In [10]:
from sklearn.ensemble import RandomForestRegressor

X_train = df_features.loc[idx_train, :]
y_train = df_zones.loc[idx_train, 'calls_daily']

model = RandomForestRegressor(n_estimators=100, n_jobs=4)
model.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=4,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

#### Пример таблицы с предсказаниями (для валидационной выборки)

In [11]:
from scipy.stats import kendalltau

X_valid = df_features.loc[idx_valid, :]
y_valid = df_zones.loc[idx_valid, 'calls_daily']
y_pred = model.predict(X_valid)

target_columns = ['calls_wd{}'.format(d) for d in range(7)]

df_valid_target = df_zones.loc[idx_valid, target_columns]
df_valid_predictions = pandas.DataFrame(collections.OrderedDict([
    (column_name, y_pred)
    for column_name in target_columns
]), index=idx_valid)

In [12]:
df_valid_predictions.head()

,calls_wd0,calls_wd1,calls_wd2,calls_wd3,calls_wd4,calls_wd5,calls_wd6
zone_id,,,,,,,
9698,1.311568,1.311568,1.311568,1.311568,1.311568,1.311568,1.311568
9288,2.538414,2.538414,2.538414,2.538414,2.538414,2.538414,2.538414
6802,1.263203,1.263203,1.263203,1.263203,1.263203,1.263203,1.263203
4071,0.087551,0.087551,0.087551,0.087551,0.087551,0.087551,0.087551
11039,0.187323,0.187323,0.187323,0.187323,0.187323,0.187323,0.187323


#### Оценка качества предсказаний

In [13]:
df_comparison = pandas.DataFrame({
    'target': df_valid_target.unstack(),
    'prediction': df_valid_predictions.unstack(),
})

valid_score = kendalltau(df_comparison['target'], df_comparison['prediction']).correlation
print('Validation score:', valid_score)

Validation score: 0.656881482683


#### Таблица предсказаний для теста

In [14]:
idx_test = df_zones.query('is_test == 1').index

X_test = df_features.loc[idx_test, :]
y_pred = model.predict(X_test)

df_test_predictions = pandas.DataFrame(collections.OrderedDict([
    (column_name, y_pred)
    for column_name in target_columns
]), index=idx_test)

df_test_predictions.to_csv('data/sample_submission.csv')
df_test_predictions.head()

,calls_wd0,calls_wd1,calls_wd2,calls_wd3,calls_wd4,calls_wd5,calls_wd6
zone_id,,,,,,,
79,0.027887,0.027887,0.027887,0.027887,0.027887,0.027887,0.027887
80,0.048746,0.048746,0.048746,0.048746,0.048746,0.048746,0.048746
81,0.064491,0.064491,0.064491,0.064491,0.064491,0.064491,0.064491
82,0.072073,0.072073,0.072073,0.072073,0.072073,0.072073,0.072073
83,0.086558,0.086558,0.086558,0.086558,0.086558,0.086558,0.086558


## Визуализация объектов на карте

Рисовать на карте можно при помощи библиотеки `folium` ([документация](http://python-visualization.github.io/folium/docs-v0.5.0/quickstart.html#Getting-Started)). 

In [15]:
import folium

fmap = folium.Map([55.753722, 37.620657])

# нанесем ж/д станции
for node in tagged_nodes:
    if node.tags.get('railway') == 'station':
        folium.CircleMarker([node.lat, node.lon], radius=3).add_to(fmap)

# выделим квадраты с наибольшим числом вызовов
calls_thresh = df_zones.calls_daily.quantile(.99)
for _, row in df_zones.query('calls_daily > @calls_thresh').iterrows():
    folium.features.RectangleMarker(
        bounds=((row.lat_bl, row.lon_bl), (row.lat_tr, row.lon_tr)),
        fill_color='red',
    ).add_to(fmap)

# карту можно сохранить и посмотреть в браузере
fmap.save('map_demo.html')

fmap